<a href="https://colab.research.google.com/github/caohungphu/CS114.L21/blob/main/Colab/Predict_final_exam_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>CS114.L21 - MÁY HỌC - MACHINE LEARNING</b></h1>
<h1 align="center"><b>BÀI TẬP COLAB SỐ 03 (17/05/2021)</b></h1>
 
## **1. YÊU CẦU:**


## **2. NHÓM THỰC HIỆN**
- Cao Hưng Phú - 19520214 - 19520214@gm.uit.edu.vn
- Nguyễn Thành Công - 19521294 - 19521294@gm.uit.edu.vn
- Trần Huỳnh Kỳ Anh - 19521216 - 19521216@gm.uit.edu.vn

## **3. LINK GITHUB**
- Link GitHub nhóm: https://github.com/caohungphu/CS114.L21/
- Link GitHub bài làm: [Predict_final_exam_score.ipynb](https://github.com/caohungphu/CS114.L21/blob/main/Colab/Predict_final_exam_score.ipynb)
- Link GitHub dataset: [Dataset](https://github.com/caohungphu/CS114.L21/tree/main/Colab/CSV/17_05_2021)

## **4. SOURCE CODE**

### **4.1. Nguồn tham khảo**

1. Thư viện pandas: https://pandas.pydata.org/docs/

### **4.2. Code**

#### **4.2.1. Thêm thư viện, đọc dữ liệu, kiểm tra dữ liệu**

In [3]:
# Thêm các thư viện cần thiết
import os
import numpy as np
import pandas as pd
 
# Đường dẫn thư mục dataset
dir_dataset_folder = "/content/CS114.L21/Colab/CSV/17_05_2021/"
 
# Nếu không có đường dẫn thì clone từ GitHub về
if not os.path.exists(dir_dataset_folder):
    !git clone https://github.com/caohungphu/CS114.L21.git

In [22]:
wecode_data = pd.read_csv("/content/CS114.L21/Colab/CSV/17_05_2021/wecode.csv")
score_training_data = pd.read_csv("/content/CS114.L21/Colab/CSV/17_05_2021/score_training.csv")

In [18]:
wecode_data

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,2020-10-30 10:58:11,2020-10-30 10:58:19
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,2020-10-30 11:07:09,2020-10-30 11:07:17
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,2020-10-30 11:08:01,2020-10-30 11:23:36
3,7,0,5,4,0,Compilation Error,0,100,solution-3,2,2020-10-30 11:11:41,2020-10-30 11:11:42
4,8,0,5,4,0,Compilation Error,0,100,solution-4,2,2020-10-30 11:15:06,2020-10-30 11:15:07
...,...,...,...,...,...,...,...,...,...,...,...,...
264601,266667,0,25,1796,0,SCORE,0,100,solution-3977,4,2021-04-28 09:34:20,2021-04-28 09:34:27
264602,266668,0,25,1796,0,Compilation Error,0,100,solution-3978,2,2021-04-28 09:35:07,2021-04-28 09:35:08
264603,266669,0,25,1796,1,SCORE,10000,100,solution-3979,4,2021-04-28 09:35:15,2021-04-28 09:35:21
264604,266670,0,150,255,1,SCORE,6250,100,solution-3980,2,2021-05-11 01:34:57,2021-05-11 01:35:28


In [14]:
new_wecode_data = wecode_data[['assignment_id', 'problem_id', 'user_id', 'is_final',
       'status', 'pre_score', 'coefficient']]

In [19]:
new_wecode_data

,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient
0,0,3,4,1,SCORE,10000,100
1,0,4,4,1,SCORE,10000,100
2,0,5,4,0,Compilation Error,0,100
3,0,5,4,0,Compilation Error,0,100
4,0,5,4,0,Compilation Error,0,100
...,...,...,...,...,...,...,...
264601,0,25,1796,0,SCORE,0,100
264602,0,25,1796,0,Compilation Error,0,100
264603,0,25,1796,1,SCORE,10000,100
264604,0,150,255,1,SCORE,6250,100


In [23]:
score_training_data

,id,QT,GK,TH,CK,TBM,TBTL
0,22,10,9,8.5,8.5,8.8,8.52
1,24,10,8,10,9.5,9.4,8.41
2,27,10,10,10,10,10,8.67
3,28,10,9.5,7.5,9.5,9.2,7.39
4,29,10,9.5,5,8.5,8.2,8.05
...,...,...,...,...,...,...,...
970,1857,5,4.5,10,4,6,6.03
971,1859,8,5,10,1.5,5.4,5.06
972,1862,5,6.5,1,7.5,5.1,4.71
973,1865,6.5,,3,7.5,,6.89


In [26]:
new_wecode_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264606 entries, 0 to 264605
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   assignment_id  264606 non-null  int64 
 1   problem_id     264606 non-null  int64 
 2   user_id        264606 non-null  int64 
 3   is_final       264606 non-null  int64 
 4   status         264605 non-null  object
 5   pre_score      264606 non-null  int64 
 6   coefficient    264606 non-null  object
dtypes: int64(5), object(2)
memory usage: 14.1+ MB


In [24]:
new_score_training_data = score_training_data[['id', 'CK']]

In [47]:
score_training_data.head(50)

,id,QT,GK,TH,CK,TBM,TBTL
0,22,10,9,8.5,8.5,8.8,8.52
1,24,10,8,10,9.5,9.4,8.41
2,27,10,10,10,10,10,8.67
3,28,10,9.5,7.5,9.5,9.2,7.39
4,29,10,9.5,5,8.5,8.2,8.05
5,30,10,10,9,9,9.3,8.63
6,34,,,9,6.5,,6.24
7,35,7,9.5,6,4.5,6.2,5.75
8,36,4.5,8,9,7,7.6,8.51
9,37,5.5,5.5,7.5,7.5,6.9,8.27


In [27]:
new_score_training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      975 non-null    int64 
 1   CK      974 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.4+ KB


In [42]:
new_score_training_data.replace(' ', np.NaN)
new_score_training_data.CK.value_counts()

8      104
7.5     94
7       78
8.5     73
6.5     67
6       63
5.5     61
9       61
5       60
        54
4       48
4.5     48
9.5     30
3.5     29
2.5     28
3       25
0       24
2       10
1.5      6
1        6
0.5      3
10       2
Name: CK, dtype: int64

### **5.1. Nhận xét**


### **5.2. Trả lời yêu cầu đề bài**
